In [44]:
import requests as re
import os
import pandas as pd
import numpy as np
import sqlite3
from sqlite3 import Error

four_api = os.environ['FOURSQUARE_API_KEY']
yelp_api = os.environ['YELP_API_KEY']
google_api = os.environ['GOOGLE_API_KEY']

#defining location parameters for searches
places = 'bar'
radius = '1000'
lat = '37.8044'
long = '-122.2712'


four_header_dict = {
    'Accept': 'application/json',
    'Authorization': four_api
}

yelp_header_dict = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {yelp_api}'
}

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
    except Error as e:
        print(f"SQL Error'{e}'")
        
connect = sqlite3.connect('../data/poi.db')


def data_to_sql(data, connection, table_name):
    """
    Takes a list of dictionaries containing consistently formatted data, an SQL connection, and a table_name as a string, 
    and creates a SQL table with all data inserted. Column names/dictionary keys cannot contain spaces.
    """
    #Takes the first dictionary and obtains column names and data types from its keys, changing types into their SQL counterparts
    sample = data[0]
    columns = {}
    for key in sample.keys():
        columns[key] = type(sample[key])
    for element in columns.keys():
        if columns[element] == int:
            columns[element] = 'INTEGER'
        elif columns[element] == str:
            columns[element] = 'TEXT'
        elif columns[element] == float:
            columns[element] = 'DECIMAL'
        else:
            return 'Add data type to function'
        
    #creates SQL table with columns named after dictionary keys
    create_table = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT'''
    for element in columns.keys():
        create_table += f', {element} {columns[element]}'
    create_table += ');'
    execute_query(connection, create_table)
    
    #inserts data into table
    insert = f'''
    INSERT INTO {table_name} ({", ".join(list(columns.keys()))}) VALUES ('''
    for row in data:
        row_insert = []
        for column in columns:
            row_insert.append(f'"{row[column]}"')
        row_insert = insert + ', '.join(row_insert) + ');'
        execute_query(connection, row_insert)

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [45]:
four_url = 'https://api.foursquare.com/v3/'
four_search = 'places/search'
returns = 'name%2Ccategories%2Clocation%2Crating%2Cpopularity%2Cmenu'
limit = '50'
four_params = f'?query={places}&ll={lat}%2C{long}&radius={radius}&fields={returns}&limit={limit}'
four_response = re.get(four_url + four_search + four_params, headers = four_header_dict)
four_res = four_response.json()

Parse through the response to get the POI details you want (rating, name, location, etc)

In [46]:
'''finds missing rating and popularity metrics, inserting values outside of their defined bounds (1-10 for 'rating', 0-1 for 'popularity') 
for easier filtering later
'''

for result in four_res['results']:
    if 'rating' not in result:
        result['rating'] = 11
    if 'popularity' not in result:
        result['popularity'] = 2

# creates list of dictionaries containing relevant place information
four_info = []
for result in four_res['results']:
    info = {}
    info['name'] = result['name']
    info['categories'] = result['categories'][0]['name']
    info['location'] = result['location']['formatted_address']
    info['popularity'] = result['popularity']
    info['rating'] = result['rating']
    four_info.append(info)

#creates another list of dictionaries, this one containing more detailed category information about each place
four_cats = []
for result in four_res['results']:
    cat = {}
    cat['name'] = result['name']
    try:
        cat['category_one'] = result['categories'][0]['name']
    except:
        cat['category_one'] = 'None'
    try:
        cat['category_two'] = result['categories'][1]['name']
    except:
        cat['category_two'] = 'None'
    try:
        cat['category_three'] = result['categories'][2]['name']
    except:
        cat['category_three'] = 'None'
    four_cats.append(cat)

In [47]:
four_cats[:3]

[{'name': 'Bar Shiru',
  'category_one': 'Cocktail Bar',
  'category_two': 'Whisky Bar',
  'category_three': 'Restaurant'},
 {'name': 'Bar 355',
  'category_one': 'Bar',
  'category_two': 'Restaurant',
  'category_three': 'None'},
 {'name': 'Twelve Bar',
  'category_one': 'Bar',
  'category_two': 'American Restaurant',
  'category_three': 'None'}]

In [48]:
#drops the table if it exists in order to prevent duplicate entries and to allow for easier changing of table structure
execute_query(connect, 'DROP TABLE foursquare_places')
execute_query(connect, 'DROP TABLE foursquare_categories')

SQL Error'no such table: foursquare_places'
SQL Error'no such table: foursquare_categories'


In [49]:
data_to_sql(four_info, connect, 'foursquare_places')

In [50]:
def data_to_sql2(data, connection, table_name):
    """
    Takes a list of dictionaries containing consistently formatted data, an SQL connection, and a table_name as a string, 
    and creates a SQL table with all data inserted. Column names/dictionary keys cannot contain spaces.
    """
    #Takes the first dictionary and obtains column names and data types from its keys, changing types into their SQL counterparts
    sample = data[0]
    columns = {}
    for key in sample.keys():
        columns[key] = type(sample[key])
    for element in columns.keys():
        if columns[element] == int:
            columns[element] = 'INTEGER'
        elif columns[element] == str:
            columns[element] = 'TEXT'
        elif columns[element] == float:
            columns[element] = 'DECIMAL'
        else:
            return 'Add data type to function'
        
    #creates SQL table with columns named after dictionary keys
    create_table = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT'''
    for element in columns.keys():
        create_table += f', {element} {columns[element]}'
    foreign = 'FOREIGN KEY(name) REFERENCES foursquare_places(name)'
    create_table += ');'
    execute_query(connection, create_table)
    
    #inserts data into table
    insert = f'''
    INSERT INTO {table_name} ({", ".join(list(columns.keys()))}) VALUES ('''
    for row in data:
        row_insert = []
        for column in columns:
            row_insert.append(f'"{row[column]}"')
        row_insert = insert + ', '.join(row_insert) + ');'
        execute_query(connection, row_insert)

In [51]:
data_to_sql2(four_cats, connect, 'foursquare_categories')

Put your parsed results into a DataFrame

In [52]:
four_data = pd.read_sql_query('SELECT * FROM foursquare_places', connect, index_col='id')
four_cats_data = pd.read_sql_query('SELECT * FROM foursquare_categories', connect, index_col='id')

In [53]:
four_data.head()

,name,categories,location,popularity,rating
id,,,,,
1,Bar Shiru,Cocktail Bar,"1611 Telegraph Ave, Oakland, CA 94612",0.974027,8.9
2,Bar 355,Bar,"355 19th St, Oakland, CA 94612",0.923766,7.9
3,Twelve Bar,Bar,"1001 Broadway (11th Street), Oakland, CA 94607",0.867315,6.6
4,Stinky’s Bar,Bar,"1644 Telegraph Ave (17th), Oakland, CA 94612",0.944793,11.0
5,Cafe Van Kleef,Art Gallery,"1621 Telegraph Ave (at 17th St), Oakland, CA 9...",0.979221,8.9


In [54]:
four_cats_data.head()

,name,category_one,category_two,category_three
id,,,,
1,Bar Shiru,Cocktail Bar,Whisky Bar,Restaurant
2,Bar 355,Bar,Restaurant,None
3,Twelve Bar,Bar,American Restaurant,None
4,Stinky’s Bar,Bar,None,None
5,Cafe Van Kleef,Art Gallery,None,None


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [55]:
yelp_url = 'https://api.yelp.com/v3/businesses/'
yelp_search = 'search'
yelp_params = f'?term={places}&latitude={lat}&longitude={long}&radius={radius}&limit={limit}'
yelp_response = re.get(yelp_url + yelp_search + yelp_params, headers = yelp_header_dict)
yelp_res = yelp_response.json()

Parse through your result and get POI details

In [56]:
#Parses through each result from the yelp search, running an API call each time to obtain additional information about each place

yelp_info = []
for business in yelp_res['businesses']:
    bid_id = business['id']
    bid_info = re.get(yelp_url + bid_id, headers = yelp_header_dict).json()
    info = {}
    info['name'] = business['name']
    info['categories'] = business['categories'][0]['title']
    info['location'] = ' '.join(business['location']['display_address'])
    info['popularity'] = business['review_count']
    info['rating'] = bid_info['rating']
    yelp_info.append(info)

Put your parsed results into a DataFrame

In [57]:
execute_query(connect, 'DROP TABLE yelp_places')

SQL Error'no such table: yelp_places'


In [58]:
data_to_sql(yelp_info, connect, 'yelp_places')

In [59]:
yelp_data = pd.read_sql_query('SELECT * FROM yelp_places', connect, index_col='id')

In [60]:
yelp_data.head()

,name,categories,location,popularity,rating
id,,,,,
1,Mad Oak,Bars,"135 12th St Oakland, CA 94607",713,4.0
2,Night Heron,Bars,"1780 Telegraph Ave Oakland, CA 94612",5,5.0
3,Hello Stranger,Bars,"1724 Broadway Oakland, CA 94612",156,4.0
4,First Edition,Cocktail Bars,"1915 Broadway Oakland, CA 94612",72,4.0
5,The Miranda,Cocktail Bars,"1739 Broadway Oakland, CA 94612",110,4.0


# Google (stretch)

In [61]:
google_search = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
google_params = f'keyword={places}&location={lat}%2C{long}&radius={radius}&key={google_api}'
google_response = re.get(google_search + google_params)
google_json = google_response.json()

In [62]:
#parses through the data returned from the google place search, running another api call about each place to obtain additional information

google_info = []
g_fields = 'formatted_address%2Cuser_ratings_total'

for result in google_json['results']:
    bid_id = result['place_id']
    bid_info = re.get(f'https://maps.googleapis.com/maps/api/place/details/json?fields={g_fields}&place_id={bid_id}&key={google_api}').json()
    info = {}
    info['name'] = result['name']
    info['categories'] = result['types'][0]
    info['location'] = bid_info['result']['formatted_address']
    info['popularity'] = bid_info['result']['user_ratings_total']
    info['rating'] = result['rating']
    google_info.append(info)

In [63]:
execute_query(connect, 'DROP TABLE google_places')

SQL Error'no such table: google_places'


In [64]:
data_to_sql(google_info, connect, 'google_places')

In [65]:
google_data = pd.read_sql_query('SELECT * FROM google_places', connect, index_col='id')

In [66]:
google_data.head()

,name,categories,location,popularity,rating
id,,,,,
1,Bar 355,bar,"355 19th St, Oakland, CA 94612, USA",211,4.5
2,Hello Stranger,bar,"1724 Broadway, Oakland, CA 94612, USA",337,4.3
3,Merchants' Saloon,bar,"Shasta Produce, 401 2nd St, Oakland, CA 94607,...",429,4.5
4,Somar Bar and Lounge,bar,"1727 Telegraph Ave, Oakland, CA 94612, USA",416,4.4
5,DREXL,bar,"382 19th St, Oakland, CA 94612, USA",277,4.5


Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Get the top 10 restaurants according to their rating

In [67]:
yelp10 = yelp_data.sort_values('rating', ascending = False).head(10)
foursquare10 = four_data[four_data['rating'] <= 10].sort_values('rating', ascending = False).head(10)
google10 = google_data.sort_values('rating', ascending = False).head(10)

In [68]:
foursquare10

,name,categories,location,popularity,rating
id,,,,,
46,Drake's Dealership,Beer Garden,"2325 Broadway (at 23rd St), Oakland, CA 94612",0.991932,9.3
9,Mad Oak,Cocktail Bar,"135 12th St (at Madison St), Oakland, CA 94607",0.991213,9.2
20,Telegraph Bar and Beer Garden,Beer Garden,"2318 Telegraph Ave (at 23rd St), Oakland, CA 9...",0.975767,9.0
1,Bar Shiru,Cocktail Bar,"1611 Telegraph Ave, Oakland, CA 94612",0.974027,8.9
5,Cafe Van Kleef,Art Gallery,"1621 Telegraph Ave (at 17th St), Oakland, CA 9...",0.979221,8.9
7,Drexl,Beer Bar,"382 19th St (btwn Webster & Franklin), Oakland...",0.959189,8.9
10,Viridian,Cocktail Bar,"2216 Broadway (btw Grand & 22nd), Oakland, CA ...",0.979166,8.9
17,The Port Bar,Restaurant,"2023 Broadway, Oakland, CA 94612",0.982648,8.5
44,The Punchdown,Wine Bar,"1737 Broadway, Oakland, CA 94612",0.968832,8.3


In [69]:
foursquaremissing = four_data[four_data['rating'] > 10].count()
foursquaremissing

name          21
categories    21
location      21
popularity    21
rating        21
dtype: int64

In [70]:
yelp10

,name,categories,location,popularity,rating
id,,,,,
24,Binny’s,Cocktail Bars,"532 8th St Oakland, CA 94607",42,5.0
20,The Kon Tiki Room,Tiki Bars,"1900 Telegraph Ave Oakland, CA 94612",5,5.0
41,Dinner House,Beer Bar,"329 14th St Oakland, CA 94612",23,5.0
2,Night Heron,Bars,"1780 Telegraph Ave Oakland, CA 94612",5,5.0
36,Tamarack,Cocktail Bars,"1501 Harrison St Oakland, CA 94612",10,5.0
17,Sante Adairius Oakland Arbor,Brewpubs,"460 8th St Oakland, CA 94607",5,5.0
25,19th Street Station,Pubs,"339 19th St Oakland, CA 94612",53,4.5
34,Low Bar,Cocktail Bars,"2300 Webster St Oakland, CA 94612",79,4.5
6,Bar Shiru,Cocktail Bars,"1611 Telegraph Ave Ste 100 Oakland, CA 94612",145,4.5


In [71]:
google10

,name,categories,location,popularity,rating
id,,,,,
9,Low Bar,bar,"2300 Webster St, Oakland, CA 94612, USA",93,4.8
14,Bar Shiru,bar,"1611 Telegraph Ave #100, Oakland, CA 94612, USA",210,4.8
10,Penelope,bar,"555 12th St Ste. 120, Oakland, CA 94607, USA",51,4.6
1,Bar 355,bar,"355 19th St, Oakland, CA 94612, USA",211,4.5
13,Cafe Van Kleef,bar,"1621 Telegraph Ave, Oakland, CA 94612, USA",895,4.5
12,First Edition,bar,"1915 Broadway, Oakland, CA 94612, USA",106,4.5
20,The Kon-Tiki,bar,"347 14th St #3906, Oakland, CA 94612, USA",482,4.5
7,19th Street Station,bar,"339 19th St, Oakland, CA 94612, USA",65,4.5
5,DREXL,bar,"382 19th St, Oakland, CA 94612, USA",277,4.5


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).